In [ ]:
import couchdb
import pandas as pd 
import numpy as np
import os
from dotenv import load_dotenv
import re 
import matplotlib.pyplot as plt
import seaborn as sns


## 1. Verbindung zur existierenden Datenbank

In [ ]:
load_dotenv(dotenv_path='.env')
COUCHDB_USER = os.getenv("COUCHDB_USER")
COUCHDB_PASSWORD = os.getenv("COUCHDB_PASSWORD")
COUCHDB_HOST = "localhost:5984"  
COUCHDB_URL = f"http://{COUCHDB_USER}:{COUCHDB_PASSWORD}@{COUCHDB_HOST}"
DB_NAME = 'world_factbook' 

try: 
    server = couchdb.Server(COUCHDB_URL)
    print(f"Erfolgreich verbunden mit CouchDB unter {COUCHDB_URL}")
    
    if DB_NAME in server:
        db = server[DB_NAME]
        print(f"Datenbank {DB_NAME} erfolgreich ausgewählt")
    else:
        print(f"Datenbank {DB_NAME} nicht gefunden!")
        raise LookupError(f"Datenbank {DB_NAME} nicht gefunden!")

except Exception as e:
    print(f"Fehler beim Verbinden mit CouchDB: {e}")
    raise e

### Hilfsfunktion zum Erstellen/Aktualisieren von Design-Dokumenten

In [ ]:
def sync_design_doc(db_handle, design_doc_dict):
    if not db_handle:
        print("sync_design_doc: Keine Datenbankverbindung.")
        return False
        
    doc_id = design_doc_dict['_id']
    try:
        existing_doc = db_handle.get(doc_id)
        if existing_doc:
            if existing_doc.get('views') != design_doc_dict.get('views') or \
               existing_doc.get('language') != design_doc_dict.get('language'):
                design_doc_dict['_rev'] = existing_doc['_rev'] 
                db_handle.save(design_doc_dict)
                print(f"Design-Dokument '{doc_id}' aktualisiert.")
                return True
            else:
                print(f"Design-Dokument '{doc_id}' ist bereits aktuell.")
                return False
        else:
            # Design-Dokument existiert nicht, neu erstellen
            db_handle.save(design_doc_dict)
            print(f"Design-Dokument '{doc_id}' erstellt.")
            return True
    except Exception as e:
        print(f"Fehler beim Synchronisieren des Design-Dokuments '{doc_id}': {e}")
        return False

### Hilfsfunktion zum Parsen von Zahlen aus Strings


In [ ]:
def parse_numeric_value_from_string(value_str, is_percentage=False, is_rate_per_1000=False):
    if pd.isna(value_str) or not isinstance(value_str, str):
        return np.nan # Wichtig für Pandas Operationen wie Quantilberechnung
    match = re.search(r'(-?\d+\.?\d*|-?\d*\.?\d+)', value_str)
    if match and match[0]:
        val = float(match.group(1))
        # Spezifische Anpassungen, falls nötig (hier nicht, da Quantile relativ sind)
        return val
    return np.nan

## Datenauswertung

## 1. Analyse: Wirtschaftliche Dynamik und Ungleichheit 

**Fragestellung:** Welche Länder verzeichnen ein hohes reales Wirtschaftswachstum, weisen aber gleichzeitig eine hohe Einkommensungleichheit (gemessen am Gini-Koeffizienten) auf?

**Ziel der Analyse:** Diese Analyse soll Länder identifizieren, bei denen der generierte Wohlstand möglicherweise nicht breit in der Bevölkerung ankommt oder bei denen schnelles Wirtschaftswachstum mit einer Zunahme der Ungleichheit einhergeht. Dies kann Anstöße für Diskussionen über Verteilungsgerechtigkeit und die Nachhaltigkeit von Wachstumsmodellen geben.

**Benötigte Felder:**
* `Economy: Real GDP growth rate` – Gibt die jährliche prozentuale Veränderung des realen Bruttoinlandsprodukts an.
* `Economy: Gini Index coefficient - distribution of family income` – Ein Maß für die Ungleichverteilung der Einkommen oder des Konsums innerhalb eines Landes. Ein Wert von 0 bedeutet vollkommene Gleichheit, ein Wert von 100 bedeutet vollkommene Ungleichheit.
* `Government: Country name - conventional short form` – Für die Identifikation der Länder.



### 1.1 **CouchDB-Technik: Map View**
Erstellung einer Map View namens `gdp_growth_vs_gini`.
* **Map-Funktion:** Diese JavaScript-Funktion wird für jedes relevante Dokument ausgeführt.
    1.  Sie prüft, ob die Felder für die Wachstumsrate und den Gini-Index vorhanden sind.
    2.  Sie **parst** die als String vorliegenden Werte für Wachstumsrate (z.B. aus "3.5%" wird `3.5`) und den Gini-Koeffizienten (z.B. aus "42.5" wird `42.5`) in numerische Formate. 
    3.  Bei erfolgreichem Parsen emittiert die Funktion einen **zusammengesetzten Schlüssel** `[parsed_gdp_growth, parsed_gini]` und den Ländernamen als Wert.
    4.  Dokumente, bei denen das Parsing fehlschlägt oder Felder fehlen, werden nicht emittiert.

In [ ]:
if db:
    design_doc_economic_analysis = {
        '_id': '_design/economic_analysis', 
        'language': 'javascript',
        'views': {
            'gdp_growth_vs_gini': { 
                'map': """
                    function(doc) {
                        if (doc.type === 'country' && 
                            doc["Economy: Real GDP growth rate"] && 
                            doc["Economy: Gini Index coefficient - distribution of family income"] &&
                            doc["Government: Country name - conventional short form"]) {

                            var gdp_growth_str = doc["Economy: Real GDP growth rate"];
                            var gini_str = doc["Economy: Gini Index coefficient - distribution of family income"];
                            var country_name = doc["Government: Country name - conventional short form"];

                            var parsed_gdp_growth = null;
                            var parsed_gini = null;

                            var match_growth = gdp_growth_str.match(/(-?\\d+\\.?\\d*)/);
                            if (match_growth[0]) {
                                parsed_gdp_growth = parseFloat(match_growth[0]);    
                            } 

                            var match_gini = gini_str.match(/(\\d+\\.?\\d*)/);
                            if (match_gini[0]) {
                                parsed_gini = parseFloat(match_gini[0]);
                            } 

                            emit([parsed_gdp_growth, parsed_gini], country_name);
                            
                        }
                    }
                """
            }
        }
    }
    # Design-Dokument in CouchDB speichern/aktualisieren
    sync_design_doc(db, design_doc_economic_analysis)
else:
    print("Keine Datenbankverbindung ('db'), Analyse übersprungen.")


### 1.2 **Abfrage der View aus Python:**
Die erstellte View kann dann mit `startkey`- und `endkey`-Parametern abgefragt werden, um Länder zu finden, deren Wachstumsraten und Gini-Koeffizienten in bestimmten Wertebereichen liegen. Python dient dazu, die View-Definition zu erstellen/aktualisieren, die Abfrage zu formulieren und die Ergebnisse darzustellen.

In [ ]:
wachstum_schwellenwert = 4.0
gini_schwellenwert = 40.0

print(f"\nView-Abfrage für Länder mit: \nReal GDP growth > {wachstum_schwellenwert}% UND Gini Index > {gini_schwellenwert}")
try:
    query_startkey = [wachstum_schwellenwert, gini_schwellenwert]
    query_endkey = [{}, {}] # {} -> Bis zum Maximum für beide Schlüsselkomponenten

    results = db.view(
        'economic_analysis/gdp_growth_vs_gini',
        startkey=query_startkey,
        endkey=query_endkey,
        reduce=False 
    )

    found_countries = []
    for row in results:
        found_countries.append({
            'Land': row.value,
            'Wachstumsrate (%)': row.key[0],
            'Gini-Index': row.key[1]
        })

    if found_countries:
        df_results = pd.DataFrame(found_countries)
        print(f"\n{len(df_results)} Länder gefunden, die die Kriterien erfüllen:")
        display(df_results.sort_values(by=['Wachstumsrate (%)', 'Gini-Index'], ascending=[False, False]))
    else:
        print("\nKeine Länder gefunden, die die spezifizierten Kriterien erfüllen.")
        
except Exception as e:
    print(f"Fehler beim Abfragen der View 'gdp_growth_vs_gini': {e}")


## 2. Analyse: Bildungsrendite-Paradoxon

Diese Analyse untersucht das Phänomen des "Bildungsrendite-Paradoxons". Es sollen Länder identifiziert werden, die signifikante Bildungsinvestitionen tätigen (approximiert durch Ausgaben und Ausbildungsdauer), aber gleichzeitig Indikatoren für potenziellen "Brain-Drain" oder mangelnde inlandige Perspektiven aufweisen (approximiert durch Netto-Migrationsrate und Jugendarbeitslosigkeit). 
Die Kernfrage ist: **Welche Länder zeigen ein Profil hoher Bildungswerte bei gleichzeitig Anzeichen für hohen Abwanderungsdruck?** Die Analyse nutzt serverseitiges Parsen der Rohdaten in CouchDB und eine flexible, clientseitige Kategorisierung und Filterung der Profile in Python.

**Datenbasis und relevante Felder:**
Die Analyse basiert auf folgenden Feldern aus dem CIA World Factbook Datensatz, die serverseitig aus ihren ursprünglichen String-Formaten in numerische Werte umgewandelt werden:
* `People and Society: Education expenditures` (als % des BIP)
* `People and Society: School life expectancy (primary to tertiary education) - total` (in Jahren)
* `People and Society: Net migration rate` (pro 1.000 Einwohner)
* `Economy: Youth unemployment rate (ages 15-24) - total` (als %)
* `Government: Country name - conventional short form` (zur Identifikation)
* `type`: Ein beim Import hinzugefügtes Feld zur Filterung auf Dokumente vom Typ 'country'.


### 2.1 **Methodisches Vorgehen:**

Serverseitige Datenaufbereitung (Parsing) mit CouchDB MapReduce View

Um eine saubere und numerische Datengrundlage für die Analyse in Python zu schaffen, wird eine CouchDB MapReduce View verwendet.
* **View-Name:** `parsed_education_migration_indicators` (im Design-Dokument `_design/indicator_parser`).
* **Map-Funktion (JavaScript):**
    1.  **Filterung:** Verarbeitet ausschließlich Dokumente vom `type: 'country'`.
    2.  **Existenzprüfung:** Stellt sicher, dass alle vier für die Analyse benötigten Rohfelder sowie der Ländername im Dokument vorhanden sind.
    3.  **Robustes Parsen:** Extrahiert für jeden der vier Indikatoren den ersten gefundenen numerischen Wert aus dem jeweiligen String-Feld (z.B. aus "2.9% of GDP (2020 est.)" wird die Zahl `2.9`). Die Funktion `parseNum` innerhalb der Map-Funktion nutzt dazu reguläre Ausdrücke (`String.match()`) und `parseFloat()`. "NA"-Werte oder nicht erfolgreich parsierbare Strings resultieren in `null`.
    4.  **Emission:** Wenn alle vier Indikatoren erfolgreich zu Zahlen geparst werden konnten, emittiert die Funktion:



In [ ]:
if db: 
    map_function_js_parser = """
    function(doc) {
        if (doc.type === 'country' &&
            doc["Government: Country name - conventional short form"] &&
            doc["People and Society: Education expenditures"] &&
            doc["People and Society: School life expectancy (primary to tertiary education) - total"] &&
            doc["People and Society: Net migration rate"] &&
            doc["Economy: Youth unemployment rate (ages 15-24) - total"]) {

            var country_name = doc["Government: Country name - conventional short form"];
            var edu_exp_str = doc["People and Society: Education expenditures"];
            var school_exp_str = doc["People and Society: School life expectancy (primary to tertiary education) - total"];
            var migration_str = doc["People and Society: Net migration rate"];
            var youth_unempl_str = doc["Economy: Youth unemployment rate (ages 15-24) - total"];

            function parseNum(strVal) {
                if (strVal === "NA" || strVal === null || typeof strVal === 'undefined') return null;
                if (typeof strVal === 'number') return strVal;
                if (typeof strVal === 'string') {
                    var match = strVal.match(/(-?\\d+\\.?\\d*|-?\\d*\\.?\\d+)/);
                    if (match && match[0]) return parseFloat(match[0]);
                }
                return null; 
            }

            var num_edu_expenditures = parseNum(edu_exp_str);
            var num_school_expectancy = parseNum(school_exp_str);
            var num_migration = parseNum(migration_str);
            var num_youth_unemployment = parseNum(youth_unempl_str);

            if (num_edu_expenditures !== null && num_school_expectancy !== null && num_migration !== null && num_youth_unemployment !== null) {
                emit(country_name, { 
                    Bildungsausgaben: num_edu_expenditures,
                    Schullebensdauer: num_school_expectancy,
                    Migration: num_migration,
                    Jugendarbeitslosigkeit: num_youth_unemployment
                });
            }
        }
    }
    """
    design_doc_parser = {
        '_id': '_design/indicator_parser', 
        'language': 'javascript',
        'views': {
            'parsed_education_migration_indicators': {
                'map': map_function_js_parser
            }
        }
    }
    sync_design_doc(db, design_doc_parser)
else:
    print("Keine Datenbankverbindung.")

### 2.2 **Clientseitige Kategorisierung, Filterung und Visualisierung (Python)**

**Definition von Schwellenwerten für Kategorien:**
    
- Für jeden der vier numerischen Indikatoren werden Schwellenwerte definiert, um die Länder in diskrete Kategorien ("Niedrig", "Mittel", "Hoch" bzw. spezifischere Bezeichnungen für Migration) einzuteilen.

**Filterung des "Paradoxon"-Profils:**
- Es wird ein spezifisches "Paradoxon"-Profil definiert, z.B. Länder mit hohen Bildungsausgaben, langer Schullebenserwartung, stark negativer Migrationsrate und hoher Jugendarbeitslosigkeit.
- Der Pandas DataFrame `df_parsed` wird nach dieser exakten Kombination von Kategorien gefiltert.


In [ ]:
# Daten aus der View abrufen 
print("\nGeparste Indikatordaten CouchDB-View...")
all_countries_parsed_data = []
try:
    results = db.view(
        'indicator_parser/parsed_education_migration_indicators',
    )
    for row in results:
        data = row.value
        data['country'] = row.key 
        all_countries_parsed_data.append(data)
    
    if not all_countries_parsed_data:
        print("Keine Daten aus der View 'parsed_education_migration_indicators' erhalten.")
    else:
        print(f"{len(all_countries_parsed_data)} Länderdatensätze aus View geladen.")
except Exception as e:
    print(f"Fehler beim Abrufen der View-Daten: {e}")
    all_countries_parsed_data = []

# Definition von Schwellenwerten 
if all_countries_parsed_data:
    df_parsed = pd.DataFrame(all_countries_parsed_data)
    
    # Bildungsausgaben (% des BIP)
    SCHWELLE_BILDUNG_NIEDRIG = 4.0
    SCHWELLE_BILDUNG_HOCH = 6.0
    
    # Schullebenserwartung (Jahre)
    SCHWELLE_SCHULLEBEN_KURZ = 8.0
    SCHWELLE_SCHULLEBEN_LANG = 15.0
    
    # Netto-Migrationsrate (pro 1.000 Einwohner)
    SCHWELLE_MIGRATION_NEGATIV = -1.0
    SCHWELLE_MIGRATION_AUSGEGLICHEN_MAX = 0.0    

    # Jugendarbeitslosigkeit (%)
    SCHWELLE_JUGENDARBEITSLOSIGKEIT_NIEDRIG = 7.0
    SCHWELLE_JUGENDARBEITSLOSIGKEIT_HOCH = 13.0

    print("\nSchwellenwerte für Kategorisierung:\n")
    print(f"  Bildungsausgaben (% BIP): \n\tNiedrig < {SCHWELLE_BILDUNG_NIEDRIG}, Mittel {SCHWELLE_BILDUNG_NIEDRIG}-{SCHWELLE_BILDUNG_HOCH}, Hoch >= {SCHWELLE_BILDUNG_HOCH}")
    print(f"  Schullebenserwartung (Jahre): \n\tKurz < {SCHWELLE_SCHULLEBEN_KURZ}, Mittel {SCHWELLE_SCHULLEBEN_KURZ}-{SCHWELLE_SCHULLEBEN_LANG}, Lang >= {SCHWELLE_SCHULLEBEN_LANG}")
    print(f"  Netto-Migrationsrate (pro 1000): \n\tNegativ < {SCHWELLE_MIGRATION_NEGATIV}, Ausgeglichen {SCHWELLE_MIGRATION_NEGATIV} bis {SCHWELLE_MIGRATION_AUSGEGLICHEN_MAX}, Positiv > {SCHWELLE_MIGRATION_AUSGEGLICHEN_MAX}")
    print(f"  Jugendarbeitslosigkeit (%): \n\tNiedrig < {SCHWELLE_JUGENDARBEITSLOSIGKEIT_NIEDRIG}, Mittel {SCHWELLE_JUGENDARBEITSLOSIGKEIT_NIEDRIG}-{SCHWELLE_JUGENDARBEITSLOSIGKEIT_HOCH}, Hoch >= {SCHWELLE_JUGENDARBEITSLOSIGKEIT_HOCH}")

    # Kategorisierung
    def categorize_value(value, min_value, max_value, category_low="Niedrig", category_mid="Mittel", category_high="Hoch"):
        if pd.isna(value): return "Unbekannt"
        if value < min_value: return category_low
        if value >= max_value: return category_high
        return category_mid

    df_parsed['bildungsausgaben_kat'] = df_parsed['Bildungsausgaben'].apply(lambda x: categorize_value(x, SCHWELLE_BILDUNG_NIEDRIG, SCHWELLE_BILDUNG_HOCH))
    df_parsed['schullebenserwartung_kat'] = df_parsed['Schullebensdauer'].apply(lambda x: categorize_value(x, SCHWELLE_SCHULLEBEN_KURZ, SCHWELLE_SCHULLEBEN_LANG, category_low="Kurz", category_high="Lang"))

    def categorize_migration(value, strong_negative, neutral_balance_limit, category_low="Negativ", category_mid="Mittel", optimal_balance="Ausgeglichen"):
        if pd.isna(value): return "Unbekannt"
        if value < strong_negative: return category_low
        if value <= neutral_balance_limit: return category_mid
        return optimal_balance

    df_parsed['migrationsrate_kat'] = df_parsed['Migration'].apply(lambda x: categorize_migration(x, SCHWELLE_MIGRATION_NEGATIV, SCHWELLE_MIGRATION_AUSGEGLICHEN_MAX))
    df_parsed['jugendarbeitslosigkeit_kat'] = df_parsed['Jugendarbeitslosigkeit'].apply(lambda x: categorize_value(x, SCHWELLE_JUGENDARBEITSLOSIGKEIT_NIEDRIG, SCHWELLE_JUGENDARBEITSLOSIGKEIT_HOCH, category_low="Niedrig", category_mid="Mittel", category_high="Hoch"))

    # Filterung des "Paradoxon"-Profils
    edu_expenditure_profile = "Hoch"
    school_life_expectancy_profile = "Lang"  
    migration_profile = "Negativ"
    youth_unemployment_profile = "Hoch"

    print(f"\nLänder mit dem primären Zielprofil: \n\tBildungsausgaben='{edu_expenditure_profile}', \n\tSchullebenserwartung='{school_life_expectancy_profile}', \n\tMigration='{migration_profile}', \n\tJugendarbeitslosigkeit='{youth_unemployment_profile}'")

    df_paradox_countries = df_parsed[
        (df_parsed['bildungsausgaben_kat'] == edu_expenditure_profile) &
        (df_parsed['schullebenserwartung_kat'] == school_life_expectancy_profile) &
        (df_parsed['migrationsrate_kat'] == migration_profile) &
        (df_parsed['jugendarbeitslosigkeit_kat'] == youth_unemployment_profile)
    ]
    
    data_for_plot = None 

    if not df_paradox_countries.empty:
        print(f"\n{len(df_paradox_countries)} Länder im primären 'Paradoxon'-Profil gefunden:")
        display(df_paradox_countries[['country', 'Bildungsausgaben', 'Schullebensdauer', 'Migration', 'Jugendarbeitslosigkeit']].sort_values(by=['Migration', 'Jugendarbeitslosigkeit'], ascending=[True, False]))
        data_for_plot = df_paradox_countries      
    else:
        print("\nKeine Länder im primären 'Paradoxon'-Profil gefunden...")



### 2.3 **Grafische Darstellung:**

   -  Die Länder, die einem der identifizierten Profile entsprechen, werden in einem Streudiagramm (Scatter Plot) visualisiert.
   -  **Achsen:** Jugendarbeitslosigkeit vs. Netto-Migrationsrate.
   -  Elemente der Grafik sind ein  Titel, klare Achsenbeschriftungen, eine Legende und die Annotation einiger Länderpunkte zur besseren Lesbarkeit.

In [ ]:
if data_for_plot is not None:
    if len(data_for_plot) > 0 : 
        plt.figure(figsize=(15, 10))
        sns.scatterplot(
            data=data_for_plot,
            x='Jugendarbeitslosigkeit',
            y='Migration',
            size='Bildungsausgaben', 
            hue='Schullebensdauer',
            sizes=(100, 700),
            alpha=0.7,
            legend='full'
        )
        
        plt.title(f"Länderprofile zum Bildungsrendite-Paradoxon", fontsize=14, fontweight='bold')
        plt.xlabel('Jugendarbeitslosigkeit (%)', fontsize=12)
        plt.ylabel('Netto-Migrationsrate (pro 1000 Einwohner)', fontsize=12)
        plt.axhline(0, color='black', linestyle='--', linewidth=0.8, label='Migrationsrate = 0')
        
        # Linien für die aktuellen Schwellenwerte des gefilterten Profils
        plt.axvline(SCHWELLE_JUGENDARBEITSLOSIGKEIT_HOCH if youth_unemployment_profile == "Hoch" 
                    else (SCHWELLE_JUGENDARBEITSLOSIGKEIT_NIEDRIG if youth_unemployment_profile == "Mittel" 
                            else -np.inf) , color='red', linestyle=':', linewidth=0.7, label=f"Schwelle Jugendarbeitslosigkeit für Kategorie '{youth_unemployment_profile}'")
        plt.axhline(SCHWELLE_MIGRATION_NEGATIV if migration_profile == "Negativ" 
                    else (SCHWELLE_MIGRATION_AUSGEGLICHEN_MAX if migration_profile == "Ausgeglichen" 
                            else np.inf), color='blue', linestyle=':', linewidth=0.7, label=f"Schwelle Migration für Kategorie '{migration_profile}'")

        num_to_annotate = min(len(data_for_plot), 7)
        if num_to_annotate > 0:
            for i in range(num_to_annotate):
                row = data_for_plot.iloc[i]
                plt.text(row['Jugendarbeitslosigkeit'] + 0.3, 
                            row['Migration'] + 0.3, 
                            row['country'], fontdict={'size': 9})

        plt.legend(title='Legende', bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.,  labelspacing=2.0)
        plt.grid(True, linestyle=':', alpha=0.6)
        plt.tight_layout(rect=[0, 0, 0.80, 1]) 
        plt.show()
    else:
        print("Nicht genügend Datenpunkte (<=1) im ausgewählten Profil für einen sinnvollen Scatterplot.")